# Logical Transduction Algebra ⚙️

This notebook demonstrates how **Logical Transduction Algebra (LTA)** is realized concretely in Agentics through **Pydantic types**, **transducible functions**, and their algebraic operators.

Rather than wiring ad-hoc prompts and tools, we work with three core ingredients:

- **ATypes (Pydantic models)** that define *typed states* in our workflow  
  – e.g. `GenericInput`, `Email`, `Movie`, `Genre`, `Summary`.
- **Transducible functions** that map one type into another in an explainable, evidence-preserving way generated dynamically via operators.
- **Algebraic operators** over types and functions that let us build complex pipelines with simple expressions.

In [1]:
from pydantic import BaseModel
from typing import Optional
from agentics.core.atype import *
from agentics.core.transducible_functions import With

class GenericInput(BaseModel):
    content: Optional[str] = None


class Email(BaseModel):
    "bla bla"
    to: Optional[str] = Field(None,description="bla bla")
    subject:Optional[str]=None
    body: Optional[str]=None

class Tweet(BaseModel):
    tweet_text:Optional[str]= Field(None,description="The content of the tweet")    

class Summary(BaseModel):
    summary_text:Optional[str]= Field(None,description="A brief summary of the input text") 

### Dynamic Generation

Transducible functions can be dynamically generared using << operator between any two pydantic types

In [2]:
input = GenericInput(content=
    """Zoran Mamdani become the NYC mayor""")

write_tweet = Email << GenericInput
tweet = await write_tweet(input)
print(tweet.model_dump_json(indent=2))


AttributeError: 'tuple' object has no attribute 'model_dump_json'

### The `With(...)` Configurator 🔧

`With(...)` is a *configuration wrapper* for transducible functions.

Instead of decorating a function with `@transducible(...)`, you can attach the same configuration **at call-time / type-level**.

Conceptually:
-	@transducible(...) decorates a Python function.
-	With(SourceType, ...) decorates a type-level transduction (TargetType << SourceType).

With(...) accepts (almost) the same arguments you would pass to @transducible, for example:
- instructions="..." – custom natural language instructions for the LLM
- llm="provider/model-name" – choose the backend model
- enforce_output_type=True – strict schema enforcement on the output
- batch_size, timeout, persist_output – runtime / Map–Reduce controls
- provide_explanation=True – ask for an explanation alongside the main output
- areduce=True – enable reduce-style behavior on lists of inputs
- Crucially, With(...) accepts (almost) the same arguments you would pass to @transducible, for example:
- instructions="..." – custom natural language instructions for the LLM
- llm="provider/model-name" – choose the backend model
- enforce_output_type=True – strict schema enforcement on the output
- batch_size, timeout, persist_output – runtime / Map–Reduce controls
- provide_explanation=True – ask for an explanation alongside the main output
- areduce=True – enable reduce-style behavior on lists of inputs

In [3]:
write_mail_to_alfio = Email<< With(
    GenericInput,
    instructions="Write an email to Alfio Gliozzo",
    prompyt_template="{content}" )
news = GenericInput(content="Zoran Mandani won the Election in NYC")
mail = await write_mail_to_alfio(news)
print(mail)

TypeError: transducible() got an unexpected keyword argument 'prompyt_template'

## Compositionality: Building Complex Behavior from Simple Pieces 🧩

Compositionality is the idea that **complex systems can be built by wiring together simpler parts**, in a way where the behavior of the whole is determined by the behavior of its components and how they are connected.

In the context of Logical Transduction Algebra and Agentics, compositionality means:

- We start with **simple, typed transformations** between Pydantic models (ATypes).
- Each transformation does **one clear job** (e.g., *“turn free text into an Email”*, *“summarize an Email”*, *“classify a Movie’s genre”*).
- We then **compose** these transformations to form richer pipelines:
  - `GenericInput → Email → Summary`
  - `Movie → Genre → Recommendation`
  - `FormInput → ValidatedForm → Ticket`

In [4]:
news = GenericInput(content="Zoran Mandani won the Election in NYC, make up a story about that.")


summary_composite_1 = Summary << write_mail_to_alfio
summary = await summary_composite_1(news)
print(summary.model_dump_json(indent=2))

summary_composite_2 = Summary <<(Email<<GenericInput)
mail = await summary_composite_2(news)
print(mail.model_dump_json(indent=2))

NameError: name 'write_mail_to_alfio' is not defined

In [5]:

from agentics import AG
from pydantic import BaseModel, Field
from typing import Optional
import agentics.core.transducible_functions


class Movie(BaseModel):
    movie_name: Optional[str] = None
    description: Optional[str] = None
    year: Optional[int] = None

class Genre(BaseModel):
    genre:Optional[str] = Field(
        None,
        description="return one category,   "
                    "e.g. comedy, drama, ...")


movies = AG.from_csv(
    "data/movies.csv", atype=Movie, max_rows=2
) 
movie = movies[1]

2026-02-13 14:45:29.771 | DEBUG    | agentics.core.agentics:from_csv:831 - Importing Agentics of type Movie from CSV str


In [6]:
classify_genre= Genre << Movie
genre = await classify_genre(movie)
print((genre @ movie).model_dump_json(indent=2))

TypeError: unsupported operand type(s) for @: 'tuple' and 'Movie'

In [7]:
await (Genre << movie)

(Genre(genre='crime'), None)

In [8]:
classify_genre= Genre << With(Movie,
                            provide_explanation=True)
genre, explanation = await classify_genre(movie)
print(genre.model_dump_json(indent=2))
print(explanation.model_dump_json(indent=2))

ERROR:root:OpenAI API call failed: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************3h4A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}
ERROR:root:OpenAI API call failed: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************3h4A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}
ERROR:root:OpenAI API call failed: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-************

An unknown error occurred. Please check the details below.
Error details: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************3h4A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}
An unknown error occurred. Please check the details below.
Error details: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************3h4A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}
An unknown error occurred. Please check the de

ERROR:root:OpenAI API call failed: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************3h4A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}
ERROR:root:OpenAI API call failed: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************3h4A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}
2026-02-13 14:45:40.304 | DEBUG    | agentics.core.async_executor:execute:67 - retrying 1 state(s), attempt 2
ERROR:root:Ope

An unknown error occurred. Please check the details below.
Error details: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************3h4A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}
An unknown error occurred. Please check the details below.
Error details: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************3h4A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}
An unknown error occurred. Please check the de

ValidationError: 1 validation error for Explanation
relevant_source_attributes
  Input should be a valid dictionary [type=dict_type, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.12/v/dict_type